# Fuel Moisture Scarios for Testing

*Jonathon Hirschi*


The purpose of this notebook is to generate fuel moisture scenarios for  model testing.

In [ ]:
# both can change
# Environment
import numpy as np, random
import pandas as pd
from MesoPy import Meso
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_raws, retrieve_raws, format_precip, fixnan, create_synthetic_data, model_decay

meso_token="4192c18707b848299783d59a9317c6e1"
m=Meso(meso_token)

In [ ]:
random.seed(123)
np.random.seed(123)

In [ ]:
# Function to simulate moisture data and equilibrium for model testing
def create_synthetic_data(days=20,power=4,data_noise=0.02,process_noise=0.0,DeltaE=0.0):
    hours = days*24
    h2 = int(hours/2)
    hour = np.array(range(hours))
    day = np.array(range(hours))/24.

    # artificial equilibrium data
    E = np.power(np.sin(np.pi*day),power) # diurnal curve
    E = 0.05+0.25*E
    # FMC free run
    m_f = np.zeros(hours)
    m_f[0] = 0.1         # initial FMC

    for t in range(hours-1):
        m_f[t+1] = max(0.,model_decay(m_f[t],E[t])  + random.gauss(0,process_noise) )
    data = m_f + np.random.normal(loc=0,scale=data_noise,size=hours)
    E = E + DeltaE    
    return E,m_f,data,hour,h2,DeltaE

In [ ]:
E,m_f,data,hour,h2,DeltaE = create_synthetic_data(days=20,power=4,data_noise=0.01,process_noise=0.0,DeltaE=0.0) 

In [ ]:
print(E.mean())
print(m_f.mean())
print(data.mean())

In [ ]:
hours = 20*24
rain = np.zeros(hours)

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(hour,E,linestyle='--',c='r',label='Equilibrium')
plt.plot(hour,m_f,linestyle='-',c='k',label='10-h fuel truth')
plt.scatter(hour[:h2],data[:h2],c='b',label='10-h fuel data')
plt.title('Synthetic data')
plt.xlabel('Time (hours)')
plt.ylabel('Fuel moisture content (%)')
plt.plot(hour,rain,linestyle='-',c='b',label='RTMA rain (mm/h)')
plt.legend()

In [ ]:
days = 20
hours = days*24
h2 = int(hours/2)
hour = np.array(range(hours))
day = np.array(range(hours))/24.
E = np.power(np.sin(np.pi*day),4)
plt.plot(hour, E)
E = 0.05+0.25*E
plt.plot(hour, E)

In [ ]:
days = 5
hours = days*24
h2 = int(hours/2)
hour = np.array(range(hours))
day = np.array(range(hours))/24.


rh = 5*np.sin(4*np.pi*day)+20
print(rh.mean())
print(rh.var())
print(rh.max())
print(rh.min())
plt.plot(hour, rh)
plt.plot(hour[:24], rh[:24])

In [ ]:
# compute relative humidity
t2 = np.array(rtma['temp']) # surface temp
rh = 100*np.exp(17.625*243.04*(td - t2) / (243.04 + t2 - 273.15) / (243.0 + td - 273.15))
Ed = 0.924*rh**0.679 + 0.000499*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh))
Ew = 0.618*rh**0.753 + 0.000454*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh)))

In [ ]:
E,m_f,data,hour,h2,DeltaE = create_synthetic_data(days=20,power=4,data_noise=0.01,process_noise=0.0,DeltaE=0.1) 